# without augmentation

In [1]:
import pickle
import pandas as pd
import itertools
from collections import Counter
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string
nltk.download('stopwords')
from tqdm.auto import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maenz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from datasets import load_dataset
dataset = load_dataset("go_emotions", "raw")

Found cached dataset go_emotions (C:/Users/maenz/.cache/huggingface/datasets/go_emotions/raw/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'],
        num_rows: 211225
    })
})

In [4]:
import copy as cp

In [5]:
df=pd.DataFrame.from_dict(dataset["train"][0:1000])

In [6]:
df.iloc[0:5]['text']

0                                      That game hurt.
1     >sexuality shouldn’t be a grouping category I...
2       You do right, if you don't care then fuck 'em!
3                                   Man I love reddit.
4    [NAME] was nowhere near them, he was by the Fa...
Name: text, dtype: object

In [7]:
label_list=["admiration","amusement","anger","annoyance","approval","caring","confusion",\
            "curiosity","desire","disappointment","disapproval","disgust","embarrassment","excitement",\
            "fear","gratitude","grief","joy","love","nervousness","optimism","pride","realization","relief","remorse","sadness","surprise","neutral"]

In [8]:
positive=["admiration","amusement","approval","caring","curiosity","desire","excitement","gratitude","joy","love","optimism","pride","relief"]
negative=["anger","annoyance","confusion","disappointment","disapproval","disgust","embarrassment","fear","grief","nervousness","remorse",\
         "sadness"]

In [9]:
df["label"]=0

In [10]:
for index, row in df.iterrows():
    for i in range(len(label_list)):
        if row[label_list[i]]==1:
            if label_list[i] in positive:
                df.at[index,'label']=1
            elif label_list[i] in negative:
                df.at[index,'label']=0
            else:
                df.at[index,'label']=2

In [11]:
df["label"]

0      0
1      0
2      2
3      1
4      2
      ..
995    1
996    1
997    1
998    0
999    2
Name: label, Length: 1000, dtype: int64

In [12]:
df=df[['text','label']]
# df.to_csv("my_emo_data")

### run from here

In [13]:
df=pd.read_csv("my_emo_data")

In [14]:
df_train=df.iloc[0:int(df.shape[0]*0.7)]
df_test=df.iloc[int(df.shape[0]*0.7):]

In [15]:
# from sklearn.model_selection import train_test_split
# X_train_original, X_valid_original, y_train_original, y_valid_original = train_test_split(df_train['text'], df_train['label'], test_size=0.3, random_state=0)

In [16]:
X_train_original=df_train['text']
X_valid_original=df_test['text']
y_train_original=df_train['label']
y_valid_original=df_test['label']

In [17]:
import torch
assert torch.cuda.is_available()
device = torch.device("cuda")

In [18]:
from transformers import AutoTokenizer, AdamW,BertForSequenceClassification

In [19]:
tokenizer= AutoTokenizer.from_pretrained("bert-base-uncased")

In [20]:
X_train=tokenizer(list(X_train_original),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_original)

In [21]:
X_valid=tokenizer(list(X_valid_original),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_valid=list(y_valid_original)

In [22]:
class MyData(torch.utils.data.Dataset):
    def __init__(self, token):
        self.token = token
    def __getitem__(self, id):
        return {key: val[id] for key, val in self.token.items()}
    def __len__(self):
        return len(self.token.input_ids)

In [23]:
bat_size=128

In [24]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)
valid_dataloader = DataLoader(MyData(X_valid) , batch_size=bat_size,shuffle = False)

In [25]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [26]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

F:\anaconda\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [27]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


epoch:   0%|          | 0/50 [00:00<?, ?it/s]

trainset:   0%|          | 0/6 [00:00<?, ?it/s]

validset:   0%|          | 0/3 [00:00<?, ?it/s]

epoch: 0
train loss: 1.1353267431259155
valid loss: 1.0794005393981934
valid accuracy: 0.43
——————————————————————————————————————————————


trainset:   0%|          | 0/6 [00:00<?, ?it/s]

...

validset:   0%|          | 0/3 [00:00<?, ?it/s]

epoch: 16
train loss: 0.8434813320636749
valid loss: 1.092007040977478
valid accuracy: 0.4066666666666667
——————————————————————————————————————————————


trainset:   0%|          | 0/6 [00:00<?, ?it/s]

validset:   0%|          | 0/3 [00:00<?, ?it/s]

early stop


In [28]:
print(max(valid_accuracy_list))

0.43333333333333335


## chatgpt baseline prompt

In [29]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentence: " +text}])
    return completion.choices[0].message.content

In [30]:
# x_new=[]
# y_new=[]
# for i in tqdm(range(len(X_train_original))):
#     count=0
#     while(count!=1):
#         try:
#             ans=rephrase_by_chatgpt(X_train_original[i])
#             x_new.append(ans)
#             y_new.append(y_train_original[i])
#             count=1  
#         except:
#             time.sleep(0.5)

In [31]:
# base_dict={"text":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
# new_df.to_csv("chatgpt_new_data_baseline")
new_df=pd.read_csv("chatgpt_new_data_baseline")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [32]:
model_base = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model_base.to(device)
optimizer = AdamW(model_base.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [33]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model_base.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model_base.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model_base(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model_base.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model_base(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model_base.state_dict(), "my_model_base")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model_base.state_dict(), "my_model_base")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 0
train loss: 1.0968098640441895
valid loss: 1.093601147333781
valid accuracy: 0.41333333333333333
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

...

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 18
train loss: 0.8001985929229043
valid loss: 1.1783377726872761
valid accuracy: 0.43
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

early stop


In [34]:
print(max(valid_accuracy_list))

0.47333333333333333


## chatgpt with downstream areaprompt

In [25]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt2(text,label):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    if(label==0):
        tone="negative"
    elif(label==1):
        tone="positive"
    else:
        tone="neutral"
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase "+text+" to be more "+ tone}])
    return completion.choices[0].message.content

In [26]:
rephrase_by_chatgpt2(X_train_original[0],y_train_original[0])

'That game was extremely painful.'

In [27]:
x_new=[]
y_new=[]
for i in tqdm(range(len(X_train_original))):
    count=0
    while(count!=1):
        try:
            ans=rephrase_by_chatgpt2(X_train_original[i],y_train_original[i])
            x_new.append(ans)
            y_new.append(y_train_original[i])
            count=1  
        except:
            time.sleep(0.5)

  0%|                                                                                          | 0/700 [00:00<…

In [28]:
base_dict={"text":x_new,"label":y_new}
new_df=pd.DataFrame.from_dict(base_dict)
new_df.to_csv("chatgpt_new_data_with_downstream_requirement")
new_df=pd.read_csv("chatgpt_new_data_with_downstream_requirement")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [29]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [30]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 0
train loss: 1.096484043381431
valid loss: 1.0803799629211426
valid accuracy: 0.41333333333333333
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

...

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 19
train loss: 0.6861298761584542
valid loss: 1.2480491002400715
valid accuracy: 0.45
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

early stop


In [31]:
print(max(valid_accuracy_list))

0.5


### chatgpt with downstream knowledge trial 2

In [42]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt3(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentences for  emotion classification: " +text}])
    return completion.choices[0].message.content

In [43]:
rephrase_by_chatgpt3(X_train_original[0])

'That game was painful.'

In [44]:
# x_new=[]
# y_new=[]
# for i in tqdm(range(len(X_train_original))):
#     count=0
#     while(count!=1):
#         try:
#             ans=rephrase_by_chatgpt3(X_train_original[i])
#             x_new.append(ans)
#             y_new.append(y_train_original[i])
#             count=1  
#         except:
#             time.sleep(0.5)

In [45]:
# base_dict={"text":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
# new_df.to_csv("chatgpt_new_data_with_downstream_trial2")
new_df=pd.read_csv("chatgpt_new_data_with_downstream_trial2")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [46]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [47]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 0
train loss: 1.10010196945884
valid loss: 1.0793220202128093
valid accuracy: 0.42
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

...

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 9
train loss: 1.0384496992284602
valid loss: 1.0700571139653523
valid accuracy: 0.43333333333333335
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

early stop


In [48]:
print(max(valid_accuracy_list))

0.45


## SOURCE OF TEXT

In [49]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt4(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentences from reddit: " +text}])
    return completion.choices[0].message.content

In [50]:
rephrase_by_chatgpt4(X_train_original[1])

'Sexuality should not be considered as a grouping category. However, it differentiates you from others, so in my opinion, it can be defined as a "grouping" characteristic.'

In [51]:
# x_new=[]
# y_new=[]
# for i in tqdm(range(len(X_train_original))):
#     count=0
#     while(count!=1):
#         try:
#             ans=rephrase_by_chatgpt4(X_train_original[i])
#             x_new.append(ans)
#             y_new.append(y_train_original[i])
#             count=1  
#         except:
#             time.sleep(0.5)

In [52]:
# base_dict={"text":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
# new_df.to_csv("chatgpt_new_data_with_source")
new_df=pd.read_csv("chatgpt_new_data_with_source")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [53]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [54]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 0
train loss: 1.0929602492939343
valid loss: 1.0870742797851562
valid accuracy: 0.41333333333333333
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

...

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 9
train loss: 1.005877440625971
valid loss: 1.081159512201945
valid accuracy: 0.43666666666666665
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

early stop


In [55]:
print(max(valid_accuracy_list))

0.44333333333333336


### SOURCE OF TEXT with requirement

In [56]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt5(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentences from Reddit but keep their emotion: " +text}])
    return completion.choices[0].message.content

In [57]:
rephrase_by_chatgpt5(X_train_original[1])

'It\'s just not fair to group people based on their sexuality. It\'s a defining characteristic that separates individuals from others, and that\'s why it fits the criteria for being a "grouping" category.'

In [58]:
# x_new=[]
# y_new=[]
# for i in tqdm(range(len(X_train_original))):
#     count=0
#     while(count!=1):
#         try:
#             ans=rephrase_by_chatgpt5(X_train_original[i])
#             x_new.append(ans)
#             y_new.append(y_train_original[i])
#             count=1  
#         except:
#             time.sleep(0.5)

In [59]:
# base_dict={"text":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
# new_df.to_csv("chatgpt_new_data_with_source_req")
new_df=pd.read_csv("chatgpt_new_data_with_source_req")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [60]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 0
train loss: 1.1004973324862393
valid loss: 1.0866859356562297
valid accuracy: 0.42
——————————————————————————————————————————————


...

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 20
train loss: 0.8197915987534956
valid loss: 1.1773096720377605
valid accuracy: 0.44
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

early stop


In [61]:
print(max(valid_accuracy_list))

0.49


## extra trial

In [62]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt6(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentences from Reddit but but keep the information that may useful for emotion classification:: " +text}])
    return completion.choices[0].message.content

In [63]:
rephrase_by_chatgpt6(X_train_original[1])

'IMO, sexuality does fit the definition of "grouping". It sets individuals apart from others and makes them different. However, I believe that it shouldn\'t be used as a category for grouping people.'

In [64]:
# x_new=[]
# y_new=[]
# for i in tqdm(range(len(X_train_original))):
#     count=0
#     while(count!=1):
#         try:
#             ans=rephrase_by_chatgpt6(X_train_original[i])
#             x_new.append(ans)
#             y_new.append(y_train_original[i])
#             count=1  
#         except:
#             time.sleep(0.5)

In [65]:
# base_dict={"text":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
# new_df.to_csv("chatgpt_new_data_extra")
new_df=pd.read_csv("chatgpt_new_data_extra")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [66]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

KeyboardInterrupt: 

In [ ]:
print(max(valid_accuracy_list))

### add known knowledge length we use

In [69]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt7(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentences to around 64 words without changing emotion: " +text}])
    return completion.choices[0].message.content

In [70]:
rephrase_by_chatgpt7(X_train_original[1])

"In my opinion, sexuality is a factor that distinguishes individuals from each other, and therefore, it should be considered a form of categorization. However, I also believe that this categorization should not lead to discrimination or marginalization of any particular group. Sexuality is a personal characteristic, and it should not define or limit a person's worth or ability to contribute to society."

In [71]:
# x_new=[]
# y_new=[]
# for i in tqdm(range(len(X_train_original))):
#     count=0
#     while(count!=1):
#         try:
#             ans=rephrase_by_chatgpt7(X_train_original[i])
#             x_new.append(ans)
#             y_new.append(y_train_original[i])
#             count=1  
#         except:
#             time.sleep(0.5)

  0%|                                                                                          | 0/700 [00:00<…

In [72]:
# base_dict={"text":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
# new_df.to_csv("chatgpt_new_data_64")
new_df=pd.read_csv("chatgpt_new_data_64")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [73]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 0
train loss: 1.0951975258913906
valid loss: 1.0794018507003784
valid accuracy: 0.41333333333333333
——————————————————————————————————————————————


...

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 11
train loss: 0.8868928280743685
valid loss: 1.1352280378341675
valid accuracy: 0.47333333333333333
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

early stop


In [74]:
print(max(valid_accuracy_list))

0.49333333333333335


## emotion

In [25]:
import os
import openai
import time
from auto_tqdm import tqdm
def chatgpt_to_spain(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please translate this sentence to Spanish while retaining the emotion:"+text}])
    return completion.choices[0].message.content
def chatgpt_to_english(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please translate this sentence to English:"+text}])
    return completion.choices[0].message.content

In [26]:
# x_new=[]
# y_new=[]
# for i in tqdm(range(len(X_train_original))):
#     count=0
#     while(count!=1):
#         try:
#             temp=chatgpt_to_spain(X_train_original[i])
#             ans=chatgpt_to_english(temp)
#             x_new.append(ans)
#             y_new.append(y_train_original[i])
#             count=1  
#         except:
#             time.sleep(0.5)

In [27]:
# base_dict={"text":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
# new_df.to_csv("emotion_back_translate")
new_df=pd.read_csv("emotion_back_translate")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=64,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [31]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 3,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [34]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 0
train loss: 0.8489940545775674
valid loss: 1.1282723744710286
valid accuracy: 0.43333333333333335
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

...

validset:   0%|                                                                                  | 0/3 [00:00<…

epoch: 12
train loss: 0.9054163315079429
valid loss: 1.1347046693166096
valid accuracy: 0.43333333333333335
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/3 [00:00<…

early stop


In [35]:
print(max(valid_accuracy_list))

0.49666666666666665
